In [1]:
import numpy as np
import random
from gymnasium.spaces import Box, Space
from poke_env.player import Gen9EnvSinglePlayer, RandomPlayer
from poke_env.environment.abstract_battle import AbstractBattle
#from poke_env.player

In [3]:
from poke_env import AccountConfiguration, Player, ServerConfiguration
from poke_env.teambuilder.teambuilder import Teambuilder


class QTable():
    def __init__(self, action_space_size) -> None:
        self.actions = [0] * action_space_size 
        self.table = {}
        
    def insert_observation(self, obs: np.ndarray):
        s = np.array2string(obs)
        
        self.table[s] = self.actions.copy()

class QPlayer(Gen9EnvSinglePlayer):
    def __init__(self, opponent: Player | str | None, account_configuration: AccountConfiguration | None = None, *, avatar: int | None = None, battle_format: str | None = None, log_level: int | None = None, save_replays: bool | str = False, server_configuration: ServerConfiguration | None = None, start_listening: bool = True, start_timer_on_battle_start: bool = False, ping_interval: float | None = 20, ping_timeout: float | None = 20, team: str | Teambuilder | None = None, start_challenging: bool = True):
        super().__init__(opponent, account_configuration, avatar=avatar, battle_format=battle_format, log_level=log_level, save_replays=save_replays, server_configuration=server_configuration, start_listening=start_listening, start_timer_on_battle_start=start_timer_on_battle_start, ping_interval=ping_interval, ping_timeout=ping_timeout, team=team, start_challenging=start_challenging)
        
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )
        
    def

    def embed_moves(self, battle: AbstractBattle):
        embedding = []
        for move in battle.available_moves:
            embedding += [move.base_power, move.type, move.category]

        return embedding

    def embed_pokemon(self, battle: AbstractBattle):
        embedding = []

        embedding += battle.active_pokemon.base_stats
        embedding += battle.active_pokemon.current_hp
        embedding += battle.active_pokemon.type

        return embedding

    def embed_battle(self, battle: AbstractBattle):
        embedding = []
        embedding += self.embed_pokemon(battle)
        embedding += self.embed_moves(battle)
        embedding += [battle.opponent_active_pokemon]
        
        return embedding

In [2]:
second_player = RandomPlayer()

p1 = RandomPlayer()

await p1.battle_against(second_player, n_battles=1)

frosmoth (pokemon object) [Active: True, Status: None]
arcaninehisui (pokemon object) [Active: True, Status: None]
flamigo (pokemon object) [Active: True, Status: None]
blastoise (pokemon object) [Active: True, Status: None]
komala (pokemon object) [Active: True, Status: None]
flamigo (pokemon object) [Active: True, Status: None]
komala (pokemon object) [Active: True, Status: FNT]
flamigo (pokemon object) [Active: True, Status: None]
arcaninehisui (pokemon object) [Active: True, Status: None]
tropius (pokemon object) [Active: True, Status: FNT]
arcaninehisui (pokemon object) [Active: True, Status: None]
mamoswine (pokemon object) [Active: True, Status: None]
arcaninehisui (pokemon object) [Active: True, Status: None]
mamoswine (pokemon object) [Active: True, Status: None]
dragonite (pokemon object) [Active: True, Status: None]
mamoswine (pokemon object) [Active: True, Status: None]
dragonite (pokemon object) [Active: True, Status: None]
mamoswine (pokemon object) [Active: True, Status:

2600000